This notebook is getting a closer look at the data in the DIANN dataset and seeing how zero shot generative models do on it.

400 samples in train_en, 400 in train_es


In [5]:
import os
from pathlib import Path

from dotenv import load_dotenv
import pandas as pd

print("Loaded .env:", load_dotenv("../../.env", override=True))
data_dir = Path(os.environ["PROJECT_DIR"]) / "data" / "host" / "diann_2023"

Loaded .env: True


In [34]:
import json

train_en = json.load(open(data_dir / "train_t1_en.json"))
train_es = json.load(open(data_dir / "train_t1_es.json"))

tokens = train_en[0]["tokens"]
labels = train_en[0]["value"]

list(zip(tokens, labels))

[('Objectives', 'O'),
 ('.', 'O'),
 ('The', 'O'),
 ('present', 'O'),
 ('research', 'O'),
 ('aims', 'O'),
 ('to', 'O'),
 ('describe', 'O'),
 ('the', 'O'),
 ('knowledge', 'O'),
 ('of', 'O'),
 ('elementary', 'O'),
 ('school', 'O'),
 ('teachers', 'O'),
 ('about', 'O'),
 ('attention', 'B-DIS'),
 ('deficit', 'I-DIS'),
 ('hyperactivity', 'I-DIS'),
 ('disorder', 'I-DIS'),
 ('(', 'O'),
 ('ADHD', 'B-DIS'),
 (')', 'O'),
 ('through', 'O'),
 ('interviews', 'O'),
 ('about', 'O'),
 ('general', 'O'),
 ('information', 'O'),
 (',', 'O'),
 ('symptoms/diagnosis', 'O'),
 ('and', 'O'),
 ('treatment', 'O'),
 (',', 'O'),
 ('in', 'O'),
 ('addition', 'O'),
 ('to', 'O'),
 ('perceived', 'O'),
 ('self', 'O'),
 ('-', 'O'),
 ('efficacy', 'O'),
 ('.', 'O'),
 ('Methods', 'O'),
 ('.', 'O'),
 ('A', 'O'),
 ('descriptive', 'O'),
 ('cross', 'O'),
 ('-', 'O'),
 ('sectional', 'O'),
 ('population', 'O'),
 ('study', 'O'),
 ('was', 'O'),
 ('carried', 'O'),
 ('out', 'O'),
 (',', 'O'),
 ('involving', 'O'),
 ('62', 'O'),
 ('teache

In [62]:
len(train_en), len(train_es)

(400, 400)

# Experimenting with r1

In [2]:
from transformers import pipeline


model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

pl = pipeline("text-generation", model=model_name, device=0)

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Device set to use cuda:0


In [15]:
" ".join(train_en[0]["tokens"])

'Objectives . The present research aims to describe the knowledge of elementary school teachers about attention deficit hyperactivity disorder ( ADHD ) through interviews about general information , symptoms/diagnosis and treatment , in addition to perceived self - efficacy . Methods . A descriptive cross - sectional population study was carried out , involving 62 teachers from public schools in the municipality of Sabaneta . The teachers were evaluated by the Spanish adaptation of the Knowledge of Attention Deficit Hyperactivity Disorder ( KADDS ) , an estimation scale composed of 36 items of three alternative answers ( true , false and I do not know ) . Results . Teachers correctly answered a little less than half of the items in the total questionnaire ( 48 .52% ) . Specifically , it was in the subscale of dymptoms/diagnosis in which they presented more hits with 69 .35% , followed by the treatment subscale ( 45 .30% ) , and finally the general information ( 38 .60% ) . Conclusions 

In [28]:
p1 = "You are a expert medical assistant that can identify disabilities from medical abstracts. You will be presented with abstracts of "
p1 += "biomedical articles related to rare diseases. You will need to identify the disabilities present in the article. A disability is define as "
p1 += "a physical or mental condition that limits a person's movements, senses, or activities. \n"
p1 += "After carefully reading the article, "
p1 += "list the disabilities present in the article. If there are multiple disabilities, list them all. If there are multiple instances of the same disability, list them as they appear in the article. If you cannot identify any disabilities, respond with 'No disabilities identified'."
p1 += "\n\nExample Abstract:\n\n"
p1 += "Objectives . The present research aims to describe the knowledge of elementary school teachers about attention deficit hyperactivity disorder ( ADHD ) through interviews about general information , symptoms/diagnosis and treatment , in addition to perceived self - efficacy . Methods . A descriptive cross - sectional population study was carried out , involving 62 teachers from public schools in the municipality of Sabaneta . The teachers were evaluated by the Spanish adaptation of the Knowledge of Attention Deficit Hyperactivity Disorder ( KADDS ) , an estimation scale composed of 36 items of three alternative answers ( true , false and I do not know ) . Results . Teachers correctly answered a little less than half of the items in the total questionnaire ( 48 .52% ) . Specifically , it was in the subscale of dymptoms/diagnosis in which they presented more hits with 69 .35% , followed by the treatment subscale ( 45 .30% ) , and finally the general information ( 38 .60% ) . Conclusions . According to the data obtained , the need for interventions on the subject is confirmed , so that this is reflected in new teaching techniques that facilitate the learning and development of children with the disorder ."
p1 += "\n\nExample Response:\n\n"
p1 += "attention deficit hyperactivity disorder\n"
p1 += "ADHD\n"

print(p1)

def generate_response(abstract):
    messages = [
        {"role": "system", "content": p1},
        {"role": "user", "content": "Abstract:\n\n"+abstract + "\n\nResponse:"},
    ]
    response = pl(messages, max_new_tokens=1000)
    return response[0]["generated_text"]

You are a expert medical assistant that can identify disabilities from medical abstracts. You will be presented with abstracts of biomedical articles related to rare diseases. You will need to identify the disabilities present in the article. A disability is define as a physical or mental condition that limits a person's movements, senses, or activities. 
After carefully reading the article, list the disabilities present in the article. If there are multiple disabilities, list them all. If there are multiple instances of the same disability, list them as they appear in the article. If you cannot identify any disabilities, respond with 'No disabilities identified'.

Example Abstract:

Objectives . The present research aims to describe the knowledge of elementary school teachers about attention deficit hyperactivity disorder ( ADHD ) through interviews about general information , symptoms/diagnosis and treatment , in addition to perceived self - efficacy . Methods . A descriptive cross -

In [33]:
print(" ".join(train_es[1]["tokens"]))

Objetivo . Analizar la asociación de competencias cognitivas en sujetos con hipoacusia unilateral severa versus sujetos con audición normal . Métodos . Participaron 40 adultos ; 20 pacientes , 10 de cada género , con hipoacusia unilateral sensorial severa y 20 sujetos sanos pareados al grupo de estudio . Las habilidades cognitivas se midieron con la batería Woodcock Muñoz - revisada y los procesos centrales auditivos con pruebas psicoacústicas monoaurales . Se realizaron gráficas de caja y prueba t de Student para muestras relacionadas con significación p ≤ 0 , 05 . Resultados . Al comparar el desempeño en las pruebas palabra filtrada y bisílabos comprimidos , se encontró diferencia estadísticamente significativa p ≤ 0 , 05 , con mayor variabilidad de respuesta en los hipoacúsicos , los cuales también tuvieron mejor desempeño cognitivo en las subpruebas inversión de números , aprendizaje visual auditivo , análisis y síntesis , formación de conceptos y palabras incompletas . Conclusione

In [29]:
generate_response(" ".join(train_es[1]["tokens"]))[-1]["content"]

"<think>\nAlright, let's tackle this problem step by step. First, I need to understand what the user is asking for. They want me to identify disabilities present in medical abstracts. A disability is defined as a physical or mental condition that limits movements, senses, or activities. The user provided an example where they identified ADHD as a disability based on the abstract.\n\nSo, the task is to go through each medical abstract, carefully read it, and list out any disabilities mentioned. If there are multiple instances of the same disability, I should include them all. If I can't find any, I should respond with 'No disabilities identified.'\n\nGiven that the abstracts are related to rare diseases, I should focus on medical terminology. I'll need to be familiar with terms like ADHD, which is a specific learning disability, or other conditions like diabetes, hypertension, or kidney disease.\n\nI'll start by reading the abstract carefully. I'll pay attention to any medical terms, di

In [22]:
train_en[1]["tokens"]

['Objective',
 'To',
 'analyse',
 'cognitive',
 'skills',
 'in',
 'patients',
 'with',
 'severe',
 'unilateral',
 'hearing',
 'loss',
 'versus',
 'those',
 'in',
 'subjects',
 'with',
 'normal',
 'hearing',
 '.',
 'Methods',
 '40',
 'adults',
 'participated',
 ':',
 '20',
 'patients',
 '(',
 '10',
 'women',
 'and',
 '10',
 'men',
 ')',
 'with',
 'severe',
 'unilateral',
 'hearing',
 'loss',
 'and',
 '20',
 'healthy',
 'subjects',
 'matched',
 'to',
 'the',
 'study',
 'group',
 '.',
 'Cognitive',
 'abilities',
 'were',
 'measured',
 'with',
 'the',
 'Spanish',
 'version',
 'of',
 'the',
 'Woodcock',
 'Johnson',
 'Battery',
 '-',
 'Revised',
 ';',
 'central',
 'auditory',
 'processing',
 'was',
 'assessed',
 'with',
 'monaural',
 'psychoacoustic',
 'tests',
 '.',
 'Box',
 'plots',
 'were',
 'drawn',
 'and',
 't',
 'tests',
 'were',
 'performed',
 'for',
 'samples',
 'with',
 'a',
 'significance',
 'of',
 'P≤',
 '.05',
 '.',
 'Results',
 'A',
 'comparison',
 'of',
 'performances',
 'on',


In [25]:
pairs = list(zip(train_es[1]["tokens"], train_es[1]["value"]))

[x for x in pairs if x[1] != "O"]

[('hipoacusia', 'B-DIS'),
 ('unilateral', 'I-DIS'),
 ('severa', 'I-DIS'),
 ('hipoacusia', 'B-DIS'),
 ('unilateral', 'I-DIS'),
 ('sensorial', 'I-DIS'),
 ('severa', 'I-DIS'),
 ('hipoacúsicos', 'B-DIS'),
 ('hipoacúsicos', 'B-DIS'),
 ('hipoacusia', 'B-DIS'),
 ('unilateral', 'I-DIS')]